# Demo 7
In this demonstration, we will learn to build ML-based classifiers to predict the likelihood of a restatement. I've provided you with a small set of financial-statement based variables that were derived based on the "FSCORE" model developed in __[Dechow, Ge, Larson, and Sloan (2011)](https://onlinelibrary.wiley.com/doi/full/10.1111/j.1911-3846.2010.01041.x)__. The FSCORE continues be used as a measure of reporting risk. The original version is based on a logistic regression. We'll see if we can improve upon that with some ML-based models.

We will proceed in the following steps:
1. Load and inspect the data. Standardize all variables to have mean 0 and standard deviation 1. Then, fit a simple logistic regression and evaluate the quality of the out-of-sample model fit.
2. Evaluate the out-of-sample performance of a Decision Tree and Random Forest using default parameters. Evaluate impact of varying tree depth in the random forest.
3. Tune the Decision Tree using `RandomizedSearchCV`. Repeat tuning with a custom evaluation function that more heavily penalizes false negatives than false positives.
4. Employ resampling and evaluate whether this improves model performance.

### Step 1: Load & Standardize the data
Let's first load the data. The data is saved in an "HDF" format, which is a highly portable data format which retains data-types (i.e., the `datetime` objects will not need to be reformatted). Note that to load this data, you'll need to install the dependency `pytables` (`!pip install pytables` should work).

In [ ]:
import pandas as pd

folder = "."
df = pd.read_hdf(f"{folder}/fscore.h5",key='key')
df.info()

There are 12 total columns in the data. Recall that `gvkey` is a firm identifier, and `datadate` is the fiscal year-end. The next 7 variables are the predictors we'll use (our X) variables. The final three variables are all different specifications of restatements:

* `BigR`: A material misstatement requiring the company to issue a press-release announcing the problem
* `AnyR`: Any restatement; includes `BigR` as well as "quiet" restatements which are simply adjusted in future financial filings
* `Intentional`: Restatements that are deemed intentional misrepresentations, as evidenced by SEC investigations

For the purpose of this demonstration, we'll attempt to predict `AnyR`, but let's look at the descriptive statistics:

In [ ]:
df[['BigR','AnyR','Intentional']].describe().transpose()

Next, we're going to standardize our data. Recall that certain machine-learning models require this, and others are more flexible. Generally, though, standardization never hurts. It retains all information in the original data but allows comparability across factors.

To standardize data to have mean 0, standard deviation 1, we can employ `sklearn`'s `StandardScaler`:

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(with_mean=True,with_std=True) # these are defaults, but says to adjust by mean and SD

XVars = ['RSSTaccruals','ChangeReceivables','ChangeInventory','PctSoftAssets','ChCashSales','chROA','ActualIssuance']

stdX = scaler.fit_transform(df[XVars])

stdX

The output of standard scaler is a numpy array; let's get these back in a dataframe.

In [ ]:
stdX_df = pd.DataFrame(stdX,columns=XVars,index=df.index)
stdX_df.describe().transpose() # confirm mean 0, std 1

Now, let's estimate a logistic regression so we have a benchmark to compare to. For all evaluations, we'll use a hold out sample. `sklearn` includes a convenient function called `train_test_split` that is used to conduct these splits. However, we're going to be a little careful with how we conduct this split.

When extracting a holdout sample, it's imperative that the holdout sample be *independent* of the data used to fit the model. In our case, we have multiple years per firm in our data:

In [ ]:
df.groupby('gvkey')['AnyR'].count().value_counts()

Specifically, we have between 1 and 14 years of data for each unique `gvkey`. Do you think MSFT in 2010 looks similar to MSFT in 2011? Of course! So, having MSFT 2010 in the training data and MSFT 2011 in the validation data would violate this independence.

Therefore, we're going to split our sample of `gvkey`s into training and testing datasets. We'll do this in two steps. First, we'll identify the gvkeys in each group. Then, we'll identify the indices that correspond to each set. This will simplify things later on.

In [ ]:
from sklearn.model_selection import train_test_split

train_gvkeys,test_gvkeys = train_test_split(df['gvkey'].unique(),test_size=0.20,random_state=123)
print(f"There are {len(train_gvkeys)} companies in the training data, and {len(test_gvkeys)} companies in the testing data.")

Now, identify the dataframe indices for each of these groups:

In [ ]:
train_idx = df[df['gvkey'].isin(train_gvkeys)].index
test_idx  = df[df['gvkey'].isin(test_gvkeys)].index
print(f"There are {len(train_idx)} observations in the training data, and {len(test_idx)} observations in the testing data.")

Now, let's see if the two datasets look comparable:

In [ ]:
stdX_df.loc[train_idx].describe().transpose()

In [ ]:
stdX_df.loc[test_idx].describe().transpose()

Not identical, but pretty similar. How about restatement rates?

In [ ]:
df.loc[train_idx,'AnyR'].mean()

In [ ]:
df.loc[test_idx,'AnyR'].mean()

Also pretty similar!

Finally, let's fit our logistic regression. For simplicity, we're going to set up four datasets that we can use in the future (will save a lot of typing): `Xtrain`, `Xtest`, `ytrain`, `ytest`

In [ ]:
from sklearn.linear_model import LogisticRegression

Xtrain = stdX_df.loc[train_idx]
Xtest  = stdX_df.loc[test_idx]

ytrain = df.loc[train_idx,'AnyR']
ytest  = df.loc[test_idx,'AnyR']

logit = LogisticRegression()
logit.fit(Xtrain,ytrain)

We've now fit our model. `sklearn` has a variety of __[evaluation metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)__ available for us to use. We're going to look at a `confusion_matrix` and a `classification_report`. Let's start with *in-sample* performance. Note that these metrics (almost) universally accept `ytrue` first, and then "`ypred`", or the predicted Y from the model:

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
confusion_matrix(ytrain,logit.predict(Xtrain))

Not very good! What does the classification report (a report that breaks down precision recall, and f1-scores by class) look like?

In [ ]:
print(classification_report(ytrain,logit.predict(Xtrain)))

What do you make of this output? Is the model biased? Accurate?

Given the poor performance in-sample, out-of-sample will likely be equally poor:

In [ ]:
print(classification_report(ytest,logit.predict(Xtest)))

### Step 2: Fit a Decision Tree and Random Forest
Next, we're going to evaluate the performance of a __[DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)__ and a __[RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)__.

Each of these models has numerous parameters, but we'll start with the defaults:

In [ ]:
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.ensemble import RandomForestClassifier as RF

dtree = DT(random_state=123)
rf = RF(random_state=123)

dtree.fit(Xtrain,ytrain)
rf.fit(Xtrain,ytrain)

print("In-sample performance of Decision Tree:\n")
print(classification_report(ytrain,dtree.predict(Xtrain)))

print("In-sample performance of Random Forest:\n")
print(classification_report(ytrain,rf.predict(Xtrain)))

Wow! Let's check out-of-sample:

In [ ]:
print("In-sample performance of Decision Tree:\n")
print(classification_report(ytest,dtree.predict(Xtest)))

print("In-sample performance of Random Forest:\n")
print(classification_report(ytest,rf.predict(Xtest)))

What happened?

The model was clearly **overfit**. This happens because we didn't set a maximum tree depth. We can see how deep the tree is with the `get_depth()` method:

In [ ]:
dtree.get_depth()

For the random forest, I can compute the average depth of each tree in the forest:

In [ ]:
import numpy as np
np.mean([tree.get_depth() for tree in rf.estimators_])

This is far too high. To illustrate, let's take our random forest and iterate over max depth from 1 to 50, counting by 2s, and we'll collect the f1-score for both in- and out-of-sample evaluations. Give the class imbalance, I'm going to go ahead and introduce some class weights here, but we'll come back to this later:

In [ ]:
from sklearn.metrics import f1_score
diag = []
for depth in range(1,51):
    print(depth)
    record = {'max_depth':depth}
    rf = RF(max_depth=depth,random_state=123,n_jobs=-1,class_weight={0:0.1,1:0.9})
    rf.fit(Xtrain,ytrain)
    record['f1_in-sample'] = f1_score(ytrain,rf.predict(Xtrain),average='macro')
    record['f1_out-of-sample'] = f1_score(ytest,rf.predict(Xtest),average='macro')
    diag.append(record)

Now we'll plot these measures.

In [ ]:
rfdiag = pd.DataFrame(diag)
rfdiag

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rfdiag['max_depth'], rfdiag['f1_in-sample'], label='In-sample F1', linestyle='-')
plt.plot(rfdiag['max_depth'], rfdiag['f1_out-of-sample'], label='Out-of-Sample F1', linestyle='--')
plt.legend()

We'd find the same thing if we did this with a Decision Tree.

We can fix this issue with Cross-validation. Cross-validation is relatively easy to implement. Let's run one instance, and then we'll repeat our loop above to show how it resolves the issue:

In [ ]:
from sklearn.model_selection import cross_validate
rf = RF(max_depth=25,random_state=123)
cv_results = cross_validate(rf,Xtrain,ytrain,cv=5,scoring='f1_macro',return_estimator=True)

The F1-scores for the 5 hold out folds are in `cv_results['test_score']`. We can compute the mean of these to evaluate performance:

In [ ]:
np.mean(cv_results['test_score'])

To see how this average compares to the validation sample, we can use the 5 different fitted models to generate predictions for our validation data, and take the average of those scores:

In [ ]:
out_of_sample_cv = [f1_score(ytest,est.predict(Xtest),average='macro') for est in cv_results['estimator']]
np.mean(out_of_sample_cv)

### Step 3: Tune the model
Now, we're going to tune our model. To save time, we're going to tune the decision tree, but the exact same principles would apply if you were tuning a random forest (you'd just have different parameters to tune).

`sklearn` offers two different classes for tuning models.

* __[GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)__ performs a grid search over all possible parameter combinations that you pass it. The benefit of this approach is that it's comprehensive. The downside is it can take forever if you choose a large set of parameters.
* __[RandomizedSeachCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)__ is very similar to the grid search, except that it randomly chooses combinations. Often, this does a good job of finding a good set of hyperparameters. We'll use the randomized search.

To use these methods, we must specify a set of parameters to try. For this, it's best to look at the documenation for the model you are tuning. Here's the link to the __[DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)__ again. You can also view the parameters for any model using the `get_params()` method:


In [ ]:
df = DT()
df.get_params()

For our purposes, we'll select a few different hyperparameters and provide some options. We do this with a dictionary, where the key corresponds to the hyperparameter and the value is a list of potential outcomes (or something else that can return a random value):

In [ ]:
dtparams = {'ccp_alpha':[0,0.001,0.01],
            'class_weight':[None,'balanced',{0:0.05,1:1},{0:1,1:2}],
            'max_depth':[2,5,10,20,30],
            'criterion':['gini','entropy','log_loss'],
            'max_features':[None,'log2','sqrt'],
            'random_state':[123]}

If we did a grid search, this would produce 540 different combinations (if my math is right). The randomized search will randomly sample various sets. Note that we include `'random_state'` as an input but don't vary it. I don't typically tune the random state, though in some scenarios that may be appropriate.

`RandomizedSearchCV` is very easy to use. It essentially acts as a trainable object. It's also highly customizable. We'll incorporate cross-validation (actually the default) and try to optimize the macro F1-score.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV as RS

# first initialize an empty Decision Tree
dtree = DT()

# Now set up the randomized search. Note that we'll optimize on 'f1', which focuses only on the "1s" (restatements)
rs = RS(dtree,param_distributions=dtparams,n_iter=50,scoring='f1',n_jobs=-1,cv=5,verbose=4,random_state=123)
results = rs.fit(Xtrain,ytrain)

To access results, we can look at `rs.cv_results_`, which is a dictionary

In [ ]:
rs.cv_results_.keys()

The `'mean_test_score'` will tell us the F1 score for the validation folds:

In [ ]:
rs.cv_results_['mean_test_score']

We can convert this dictionary to a dataframe, which makes it easier to inspect:

In [ ]:
results_df = pd.DataFrame(rs.cv_results_)
results_df.sort_values(by='mean_test_score',ascending=False)

We can also quickly look at the "best parameters" with the `best_params_` attribute:

In [ ]:
rs.best_params_

Finally, we can evaluate how this best model works on our hold out sample, treating it as if it were a standard `sklearn` object. Let's look at a confusion matrix:

In [ ]:
c = confusion_matrix(ytest,rs.predict(Xtest))
c

One way to evaluate performance is to look at how much more likely you are to correctly identify a restatement in the right column than left:

In [ ]:
(c[1,1]/c[:,1].sum()) / (c[1,0]/c[:,0].sum())

Suppose this is the metric we want to maximize. We can incorporate this into our randomized search using a custom scorer:

In [ ]:
from sklearn.metrics import make_scorer

def myscorer(ytrue,ypred): # these must be the two arguments
    return (c[1,1]/c[:,1].sum()) / (c[1,0]/c[:,0].sum())

scorer = make_scorer(myscorer,greater_is_better=True)

rs = RS(dtree,param_distributions=dtparams,n_iter=50,scoring=scorer,n_jobs=-1,cv=5,verbose=4,random_state=123)
results2 = rs.fit(Xtrain,ytrain)

Did we pick a model that performs better on this metric?

In [ ]:
print(results2.best_score_)
print(results2.best_params_)

Performance was the same, though the "best" parameters were slightly different. I think this is because what we came up with was essentially a transformation of the original metric (though one that's a little easier to understand perhaps).

In any case, we could define something more involved. Suppose we want to score with something like this, which may be how a regulator or auditor would view this problem:

* 20 points for every correctly identified restatement (c[1,1])
* -20 points for every missed restatement (c[1,0])
* -1 point for every falsely identified restatement (c[0,1])
* 2 points for every correctly identified non restatement (c[0,0])

We'll then scale that score by the count of observations:

In [ ]:
def myscorer2(ytrue,ypred): # these must be the two arguments
    c = confusion_matrix(ytrue,ypred)
    score = 20*(c[1,1] - c[1,0]) - c[0,1] + 2*c[0,0]
    return score/c.sum()

scorer2 = make_scorer(myscorer2,greater_is_better=True)

rs = RS(dtree,param_distributions=dtparams,n_iter=50,scoring=scorer2,n_jobs=-1,cv=5,verbose=4,random_state=123)
results2 = rs.fit(Xtrain,ytrain)
print(results2.best_score_)
print(results2.best_params_)

Let's look at this confusion matrix

In [ ]:
confusion_matrix(ytest,rs.predict(Xtest))

Recall is pretty good in this model (506 / (257+506)), but precision is bad (506 vs 5641+506).

Let's finish up by looking at resampling to see if that can help improve things.

### Step 4: Resampling

The last thing we will cover is resampling. Note that when resampling, you **must only resample** the training data. Your model needs to be evaluated in terms of how it performs on *real* data.

We will incorporate resampling into our parameter search in a moment, but before doing so let's quickly look at how it works. `sklearn` does not natively include resamplers, but there is a separate package, `imblearn`, that is designed to work like `sklearn`. You'll need to install if you haven't already (`!pip install imblearn`).

We'll import an oversampler and undersampler:

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE # SMOTE creates synthetic data

rus = RandomUnderSampler(random_state=123)
smote = SMOTE(random_state=123,n_jobs=-1)

Let's start with undersampling. We'll first undersample our training data (both X and y). We'll then train a `DecisionTreeClassifier`, and evaluate performance on the hold out sample:

In [ ]:
Xus,yus = rus.fit_resample(Xtrain,ytrain)
print(Xtrain.shape)
print(Xus.shape)

Let's fit our original decision tree again so we have it for comparison purposes:

In [ ]:
dtree = DT(random_state=123)
dtree.fit(Xtrain,ytrain)
print(classification_report(ytest,dtree.predict(Xtest)))

Now, we'll use the resampled data:

In [ ]:
dtree.fit(Xus,yus)
print(classification_report(ytest,dtree.predict(Xtest)))

By most metrics, overall performance declines. BUT, the model is less biased (particularly on recall). That's the idea with resampling.

Let's now check out oversampling:

In [ ]:
Xos,yos = smote.fit_resample(Xtrain,ytrain)
print(Xtrain.shape)
print(Xos.shape)

Notice the dataset grew because we added some synthetic data. Let's repeat our training:

In [ ]:
dtree.fit(Xos,yos)
print(classification_report(ytest,dtree.predict(Xtest)))

This is less biased than the original version, though more biased than undersampling (at least on recall).

As a final step, let's think about how we'd incorporate this into parameter search. Specifically, how do we deal with resampling with cross-validation, since we only want to resample the folds used for training.

The answer is that we're going to need to set up something called a **pipeline** to handle a set of instructions, which are applied, in order, for each step set of parameters we try. Fortunately, `imblearn` has set something up that makes this relatively easy to implement. We need to make two adjustments:

1. Set up a pipeline that we can tune with `RandomizedSearchCV`
2. Adjust our parameter dictionary to account for multiple steps in the pipeline (i.e. I may have parameters for both the classifier AND the resampler)

In [ ]:
from imblearn.pipeline import Pipeline

### Set Up the Pipeline ###
# The pipeline is a list of tuples, and each tuple as a name (e.g., 'smote'), and object (e.g., SMOTE())
pipeline = Pipeline([
    ('smote',SMOTE()),
    ('dtree',DT())
])

### Set up the parameter grid ###
# We'll start with the one we used before, but we need to prepend each key with 'dtree__'
params2 = {f'dtree__{k}':v for k,v in dtparams.items()}
# Now, add some parameters for smote:
params2['smote__random_state'] = [123]
params2['smote__k_neighbors'] = [2,3,4,5]

params2

Now, we can set up a randomized search like before, but use the pipeline instead of the decision tree:

In [ ]:
# Compared to above, change to pipeline and params2
rs = RS(pipeline,param_distributions=params2,n_iter=50,scoring=scorer2,n_jobs=-1,cv=5,verbose=4,random_state=123)
rs.fit(Xtrain,ytrain)
print(classification_report(ytest,rs.predict(Xtest)))
print(confusion_matrix(ytest,rs.predict(Xtest)))

And that's it! We've now learned to:

1. Implement basic ML models and generate a variety of performance reports
2. Tune these models
3. Incorporate resampling into our tuning process

While we mainly focused on decision trees, the **exact** same approach applies to essentially all __[`sklearn` classifiers](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html)__. One just needs to adjust hyperparameters to match those available for tuning in the object.